In [1]:
# ============================================================================
# CELDA 0: SETUP GOOGLE COLAB PRO+
# ============================================================================

print("="*80)
print("CONFIGURACIÓN INICIAL - GOOGLE COLAB PRO+ (TiDE h=1 CORREGIDO)")
print("="*80)

# 1. Verificar GPU
import torch
print(f"\n1. GPU CONFIGURATION:")
print(f"   CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    print(f"   ✓ GPU READY")
else:
    print("   ⚠️ WARNING: No GPU detected. Go to Runtime → Change runtime type")

# 2. Instalar paquetes faltantes
print(f"\n2. INSTALLING PACKAGES:")
print("   Installing neuralforecast...")
!pip install -q neuralforecast==1.7.4

print("   Installing optuna...")
!pip install -q optuna==3.6.1

print("   Installing arch (GARCH models)...")
!pip install -q arch==7.1.0

print("   ✓ PACKAGES INSTALLED")

# 3. Montar Google Drive
print(f"\n3. MOUNTING GOOGLE DRIVE:")
from google.colab import drive
drive.mount('/content/drive', force_remount=False)
print("   ✓ DRIVE MOUNTED")

# 4. Crear directorio de outputs
import os
output_path = '/content/drive/MyDrive/Colab_Outputs/TiDE_h1_USD_PEN_CORREGIDO'
os.makedirs(output_path, exist_ok=True)
print(f"   Output directory: {output_path}")
print("   ✓ OUTPUT DIR CREATED")

# 5. Verificar versiones
print(f"\n4. PACKAGE VERSIONS:")
import pandas as pd
import numpy as np
print(f"   Python: 3.10+")
print(f"   Pandas: {pd.__version__}")
print(f"   NumPy: {np.__version__}")
print(f"   PyTorch: {torch.__version__}")

import neuralforecast
print(f"   NeuralForecast: {neuralforecast.__version__}")

import optuna
print(f"   Optuna: {optuna.__version__}")

import arch
print(f"   ARCH: {arch.__version__}")

print("\n" + "="*80)
print("✓ SETUP COMPLETADO - TiDE h=1 CORREGIDO")
print("="*80)


CONFIGURACIÓN INICIAL - GOOGLE COLAB PRO+ (TiDE h=1 CORREGIDO)

1. GPU CONFIGURATION:
   CUDA available: True
   GPU: Tesla T4
   VRAM: 15.8 GB
   ✓ GPU READY

2. INSTALLING PACKAGES:
   Installing neuralforecast...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.4/249.4 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 831.6/831.6 kB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.3/72.3 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 66.7 MB/s eta 0:00:00
   Installing optuna...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# ============================================================================
# CELDA 0.5: UPLOAD DATA.csv
# ============================================================================

from google.colab import files
uploaded = files.upload()
# Seleccionar DATA.csv desde tu computadora

# Verificar
df_test = pd.read_csv('DATA.csv')
print(f"✓ DATA.csv loaded: {df_test.shape}")
print(f"Columns: {df_test.columns.tolist()}")
print(f"Date range: {df_test['Dates'].min()} to {df_test['Dates'].max()}")

Saving DATA.csv to DATA.csv
✓ DATA.csv loaded: (8201, 12)
Columns: ['Dates', 'PEN', 'MXN', 'CLP', 'COBRE', 'DXY', 'UST10Y', 'CPI', 'MXPE', 'RESERV', 'T_TRADE', 'Tasa_cds']
Date range: 1/01/1996 to 9/12/2024


In [3]:
# ============================================================================
# CELDA 1: IMPORTS Y CONFIGURACIÓN GLOBAL
# ============================================================================

"""
TiDE h=1 Implementation - VERSIÓN CORREGIDA
Bug fix: Cálculo de DA durante tuning ahora usa y_prev real
"""

# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from pathlib import Path
import json
import warnings
import pickle
warnings.filterwarnings('ignore')

# Sklearn imports
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

# NeuralForecast imports
from neuralforecast import NeuralForecast
from neuralforecast.models import TiDE
from neuralforecast.losses.pytorch import MAE

# Optuna
import optuna
from optuna.samplers import TPESampler

# GARCH
from arch import arch_model

# Logging
import logging

# Output directory
OUTPUT_DIR = Path('/content/drive/MyDrive/Colab_Outputs/TiDE_h1_USD_PEN_CORREGIDO')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(OUTPUT_DIR / 'tide_h1_corregido.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# ============================================================================
# CONFIGURACIÓN GLOBAL
# ============================================================================

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

H_FORECAST = 1           # One-step-ahead
MIN_TRAIN = 252          # 1 año mínimo
STEP_SIZE = 21           # 1 mes
N_HOLDOUT = 60           # 60 días holdout

# Optuna tuning
N_TRIALS = 100           # 100 trials para 10 dims
TIMEOUT = 7200           # 2 horas máximo

# Baselines para comparación
BASELINE_ARX = {'DA': 51.67, 'MASE': 0.9398}
BASELINE_NHITS = {'DA': 53.66, 'MASE': 0.9350}
BASELINE_NBEATSX = {'DA': 45.00, 'MASE': 0.9442}

logger.info("="*80)
logger.info("TiDE h=1 CORREGIDO - Configuration loaded")
logger.info("="*80)
print("✓ Celda 1 completada: Configuración cargada")

✓ Celda 1 completada: Configuración cargada


In [4]:
# ============================================================================
# CELDA 2: FXMetrics (sin cambios)
# ============================================================================

class FXMetrics:
    """Métricas para evaluación de forecasting FX"""

    @staticmethod
    def levels_to_returns(y_level: np.ndarray, y_prev_level: np.ndarray) -> np.ndarray:
        """Convierte niveles a log returns"""
        return np.log(y_level / y_prev_level)

    @staticmethod
    def directional_accuracy(y_true_ret: np.ndarray, y_pred_ret: np.ndarray) -> float:
        """Calcula Directional Accuracy"""
        correct = np.sum(np.sign(y_true_ret) == np.sign(y_pred_ret))
        total = len(y_true_ret)
        return 100.0 * correct / total if total > 0 else 0.0

    @staticmethod
    def mase(y_true_ret: np.ndarray,
             y_pred_ret: np.ndarray,
             y_train_returns: np.ndarray) -> float:
        """Calcula MASE"""
        mae_model = np.mean(np.abs(y_true_ret - y_pred_ret))
        mae_naive = np.mean(np.abs(np.diff(y_train_returns)))
        return mae_model / mae_naive if mae_naive > 0 else np.inf

    @staticmethod
    def calculate_all_metrics(y_true_level, y_pred_level, y_prev_level, y_train_returns):
        """Calcula todas las métricas"""
        y_true_ret = FXMetrics.levels_to_returns(y_true_level, y_prev_level)
        y_pred_ret = FXMetrics.levels_to_returns(y_pred_level, y_prev_level)

        da = FXMetrics.directional_accuracy(y_true_ret, y_pred_ret)
        mase_value = FXMetrics.mase(y_true_ret, y_pred_ret, y_train_returns)
        mae_ret = np.mean(np.abs(y_true_ret - y_pred_ret))
        mae_level = np.mean(np.abs(y_true_level - y_pred_level))

        return {
            'DA': da,
            'MASE': mase_value,
            'MAE_returns': mae_ret,
            'MAE_levels': mae_level
        }

    @staticmethod
    def compare_to_baseline(metrics: dict) -> dict:
        """Compara con baselines"""
        comparison = {
            'baseline_arx': BASELINE_ARX,
            'baseline_nhits': BASELINE_NHITS,
            'tide': metrics,
            'delta_vs_arx': {
                'DA': metrics['DA'] - BASELINE_ARX['DA'],
                'MASE': metrics['MASE'] - BASELINE_ARX['MASE']
            },
            'delta_vs_nhits': {
                'DA': metrics['DA'] - BASELINE_NHITS['DA'],
                'MASE': metrics['MASE'] - BASELINE_NHITS['MASE']
            }
        }

        if metrics['DA'] > BASELINE_ARX['DA'] and metrics['MASE'] < BASELINE_ARX['MASE']:
            comparison['verdict'] = "✓ SUPERIOR - Beats ARX baseline"
        elif metrics['DA'] > 50:
            comparison['verdict'] = "~ ACCEPTABLE - Above random walk"
        else:
            comparison['verdict'] = "✗ INFERIOR - Below random walk"

        return comparison

logger.info("FXMetrics class loaded")
print("✓ Celda 2 completada: FXMetrics")

✓ Celda 2 completada: FXMetrics


In [5]:
# ============================================================================
# CELDA 3: FeatureEngineer (sin cambios significativos)
# ============================================================================

class FeatureEngineer:
    """Feature engineering con anti-leakage"""

    def __init__(self):
        self.garch_model = None

    def calculate_garch_volatility(self, returns: pd.Series, window: int = 252) -> pd.Series:
        """Calcula volatilidad GARCH(1,1) con lag"""
        vol = pd.Series(index=returns.index, dtype=float)
        vol[:] = np.nan

        for i in range(window, len(returns)):
            try:
                train_returns = returns.iloc[i-window:i] * 100
                model = arch_model(train_returns, vol='Garch', p=1, q=1,
                                  mean='Constant', rescale=False)
                result = model.fit(disp='off', show_warning=False)
                forecast = result.forecast(horizon=1)
                vol.iloc[i] = np.sqrt(forecast.variance.values[-1, 0]) / 100
            except:
                if i > window:
                    vol.iloc[i] = vol.iloc[i-1]
                else:
                    vol.iloc[i] = returns.iloc[:i].std()

        return vol

    def calculate_rsi(self, prices: pd.Series, period: int = 14) -> pd.Series:
        """RSI con lag implícito"""
        delta = prices.diff()
        gain = delta.where(delta > 0, 0).rolling(window=period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
        rs = gain / loss
        rsi = 100 - (100 / (1 + rs))
        return rsi.shift(1)

    def calculate_macd(self, prices: pd.Series) -> pd.Series:
        """MACD line con lag"""
        ema12 = prices.ewm(span=12, adjust=False).mean()
        ema26 = prices.ewm(span=26, adjust=False).mean()
        macd = ema12 - ema26
        return macd.shift(1)

    def cyclical_encoding(self, df: pd.DataFrame, col: str, max_val: int) -> pd.DataFrame:
        """Encoding cíclico"""
        df[f'{col}_sin'] = np.sin(2 * np.pi * df[col] / max_val)
        df[f'{col}_cos'] = np.cos(2 * np.pi * df[col] / max_val)
        return df

    def create_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """Crea todas las features con anti-leakage"""
        logger.info("="*80)
        logger.info("FEATURE ENGINEERING - START")
        logger.info("="*80)

        df = df.copy()

        # Convertir fechas
        df['Dates'] = pd.to_datetime(df['Dates'], dayfirst=True)
        df = df.set_index('Dates')
        df = df.sort_index()

        # Target y lag
        df['PEN_target'] = df['PEN']
        df['PEN_lag_1'] = df['PEN'].shift(1)
        df['PEN_log_ret'] = np.log(df['PEN'] / df['PEN'].shift(1))

        # GARCH volatility
        logger.info("Calculating GARCH volatility...")
        df['GARCH_vol'] = self.calculate_garch_volatility(df['PEN_log_ret'].dropna())

        # RSI y MACD
        logger.info("Calculating RSI and MACD...")
        df['RSI'] = self.calculate_rsi(df['PEN'])
        df['MACD_line'] = self.calculate_macd(df['PEN'])

        # Macro/FX features con lag
        macro_cols = ['MXN', 'CLP', 'COBRE', 'MXPE']
        for col in macro_cols:
            if col in df.columns:
                df[f'{col}_ret'] = np.log(df[col] / df[col].shift(1))
                df[f'{col}_lag_1'] = df[f'{col}_ret'].shift(1)

        if 'UST10Y' in df.columns:
            df['UST10Y_diff'] = df['UST10Y'].diff()
            df['UST10Y_lag_1'] = df['UST10Y_diff'].shift(1)

        if 'DXY' in df.columns:
            df['DXY_diff'] = df['DXY'].diff()
            df['DXY_lag_1'] = df['DXY_diff'].shift(1)

        optional_cols = ['CPI', 'RESERV', 'T_TRADE', 'Tasa_cds']
        for col in optional_cols:
            if col in df.columns:
                df[f'{col}_diff'] = df[col].diff()
                df[f'{col}_lag_1'] = df[f'{col}_diff'].shift(1)

        # Variables temporales
        df['month'] = df.index.month
        df['day_of_week'] = df.index.dayofweek
        df['quarter'] = df.index.quarter

        df = self.cyclical_encoding(df, 'month', 12)
        df = self.cyclical_encoding(df, 'day_of_week', 7)
        df = self.cyclical_encoding(df, 'quarter', 4)

        # Lista de exógenas
        all_exog_features = ['GARCH_vol', 'RSI', 'MACD_line']

        for col in macro_cols:
            if f'{col}_lag_1' in df.columns:
                all_exog_features.append(f'{col}_lag_1')

        if 'UST10Y_lag_1' in df.columns:
            all_exog_features.append('UST10Y_lag_1')
        if 'DXY_lag_1' in df.columns:
            all_exog_features.append('DXY_lag_1')

        for col in optional_cols:
            if f'{col}_lag_1' in df.columns:
                all_exog_features.append(f'{col}_lag_1')

        all_exog_features.extend([
            'month_sin', 'month_cos',
            'day_of_week_sin', 'day_of_week_cos',
            'quarter_sin', 'quarter_cos'
        ])

        all_exog_features = [col for col in all_exog_features if col in df.columns]

        logger.info(f"Total exogenous features: {len(all_exog_features)}")

        # Dropear NaNs
        critical_cols = ['PEN_target', 'PEN_lag_1'] + all_exog_features
        df_clean = df[critical_cols].dropna()

        logger.info(f"After dropna: {df_clean.shape}")

        # Almacenar lista de exógenas
        df_clean.attrs['all_exog_features'] = all_exog_features

        logger.info("FEATURE ENGINEERING - COMPLETADO")
        logger.info("="*80)

        return df_clean

logger.info("FeatureEngineer class loaded")
print("✓ Celda 3 completada: FeatureEngineer")

✓ Celda 3 completada: FeatureEngineer


In [6]:
# ============================================================================
# CELDA 4: NeuralForecastFormatter (sin cambios)
# ============================================================================

class NeuralForecastFormatter:
    """Conversión wide → long + scaling"""

    def __init__(self):
        self.scaler = StandardScaler()
        self.exog_to_scale = []
        self.exog_no_scale = []
        self.fitted = False

    def format_to_long(self, df: pd.DataFrame, target_col: str, exog_features: list) -> pd.DataFrame:
        """Convierte DataFrame wide a formato long"""
        df_long = pd.DataFrame({
            'unique_id': 'PEN',
            'ds': df.index,
            'y': df[target_col].values
        })

        for exog in exog_features:
            if exog in df.columns:
                df_long[exog] = df[exog].values

        df_long = df_long.reset_index(drop=True)
        return df_long

    def fit_scaler(self, train_df: pd.DataFrame, exog_to_scale: list, exog_no_scale: list):
        """Fit scaler en train set"""
        self.exog_to_scale = exog_to_scale
        self.exog_no_scale = exog_no_scale

        if len(exog_to_scale) > 0:
            self.scaler.fit(train_df[exog_to_scale])
            self.fitted = True

    def transform_scaler(self, df: pd.DataFrame) -> pd.DataFrame:
        """Transform exógenas"""
        df_scaled = df.copy()

        if self.fitted and len(self.exog_to_scale) > 0:
            df_scaled[self.exog_to_scale] = self.scaler.transform(df[self.exog_to_scale])

        return df_scaled

    def validate_long_format(self, df: pd.DataFrame):
        """Validar formato"""
        required_cols = ['unique_id', 'ds', 'y']
        for col in required_cols:
            assert col in df.columns, f"Missing: {col}"
        assert df['unique_id'].nunique() == 1
        assert not df['y'].isna().any()
        logger.info("✓ Long format validated")

logger.info("NeuralForecastFormatter class loaded")
print("✓ Celda 4 completada: NeuralForecastFormatter")

✓ Celda 4 completada: NeuralForecastFormatter


In [7]:
# ============================================================================
# CELDA 5: TiDEBuilder (sin cambios)
# ============================================================================

class TiDEBuilder:
    """Builder para modelos TiDE"""

    @staticmethod
    def build_model(h: int,
                   input_size: int,
                   max_steps: int,
                   learning_rate: float,
                   hidden_size: int,
                   decoder_output_dim: int,
                   temporal_decoder_dim: int,
                   num_encoder_layers: int,
                   num_decoder_layers: int,
                   dropout: float,
                   batch_size: int,
                   hist_exog_list: list = None,
                   val_check_steps: int = 50,
                   early_stop_patience_steps: int = -1) -> TiDE:
        """Construye TiDE"""

        model = TiDE(
            h=h,
            input_size=input_size,
            hidden_size=hidden_size,
            decoder_output_dim=decoder_output_dim,
            temporal_decoder_dim=temporal_decoder_dim,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers,
            dropout=dropout,
            temporal_width=4,
            layernorm=True,
            hist_exog_list=hist_exog_list if hist_exog_list else [],
            futr_exog_list=[],
            stat_exog_list=[],
            max_steps=max_steps,
            learning_rate=learning_rate,
            batch_size=batch_size,
            scaler_type='standard',
            random_seed=RANDOM_STATE,
            loss=MAE(),
            val_check_steps=val_check_steps,
            early_stop_patience_steps=early_stop_patience_steps,
            num_lr_decays=2
        )

        return model

    @staticmethod
    def get_optuna_search_space() -> dict:
        """Search space para TiDE"""
        return {
            'input_size': [63, 126, 168, 252],
            'max_steps': [300, 400, 500, 600],
            'learning_rate': [1e-5, 1e-2],
            'hidden_size': [64, 128, 256, 512],
            'decoder_output_dim': [16, 32, 64, 128],
            'temporal_decoder_dim': [64, 128, 192, 256],
            'num_encoder_layers': [1, 2, 3, 4],
            'num_decoder_layers': [1, 2, 3, 4],
            'dropout': [0.0, 0.1, 0.2, 0.3, 0.5],
            'batch_size': [8, 16, 32, 64]
        }

logger.info("TiDEBuilder class loaded")
print("✓ Celda 5 completada: TiDEBuilder")

✓ Celda 5 completada: TiDEBuilder


In [8]:
# ============================================================================
# CELDA 6: TiDETuner BLINDADO (Con Persistencia y Resume)
# ============================================================================

class TiDETuner:
    """
    Tuner para TiDE con PERSISTENCIA EN DRIVE y corrección de DA.
    """

    def __init__(self, h: int = H_FORECAST):
        self.builder = TiDEBuilder()
        self.h = h

    def _calculate_da_corrected(self, y_true_levels, y_pred_levels, y_prev_levels):
        """
        ✅ CORRECCIÓN: Calcula DA usando y_prev real
        """
        if len(y_true_levels) < 5:
            # logger.warning("      Too few predictions") # Comentado para limpiar log
            return 0.0

        # ✅ CORRECTO: Usar y_prev real para calcular retornos
        y_true_ret = np.log(y_true_levels / y_prev_levels)
        y_pred_ret = np.log(y_pred_levels / y_prev_levels)

        # Filtrar valores no finitos
        valid_mask = np.isfinite(y_true_ret) & np.isfinite(y_pred_ret)
        if np.sum(valid_mask) < 5:
            return 0.0

        y_true_ret = y_true_ret[valid_mask]
        y_pred_ret = y_pred_ret[valid_mask]

        # Calcular direcciones
        dir_true = np.sign(y_true_ret)
        dir_pred = np.sign(y_pred_ret)

        # DA
        correct = np.sum(dir_true == dir_pred)
        da = 100.0 * correct / len(dir_true)

        return da

    def _objective(self, trial, train_inner_df, valid_df, full_df_features, hist_exog_list):
        """
        Función objetivo con CORRECCIÓN de DA
        """
        # (Reducimos el log inicial para no llenar la pantalla al reanudar)
        # logger.info(f"TRIAL #{trial.number} (TiDE CORREGIDO)")

        search_space = self.builder.get_optuna_search_space()

        # Sample hyperparameters
        input_size = trial.suggest_categorical('input_size', search_space['input_size'])
        max_steps = trial.suggest_categorical('max_steps', search_space['max_steps'])
        learning_rate = trial.suggest_float('learning_rate',
                                            search_space['learning_rate'][0],
                                            search_space['learning_rate'][1],
                                            log=True)
        hidden_size = trial.suggest_categorical('hidden_size', search_space['hidden_size'])
        decoder_output_dim = trial.suggest_categorical('decoder_output_dim', search_space['decoder_output_dim'])
        temporal_decoder_dim = trial.suggest_categorical('temporal_decoder_dim', search_space['temporal_decoder_dim'])
        num_encoder_layers = trial.suggest_categorical('num_encoder_layers', search_space['num_encoder_layers'])
        num_decoder_layers = trial.suggest_categorical('num_decoder_layers', search_space['num_decoder_layers'])
        dropout = trial.suggest_categorical('dropout', search_space['dropout'])
        batch_size = trial.suggest_categorical('batch_size', search_space['batch_size'])

        # Build model
        try:
            model = self.builder.build_model(
                h=self.h,
                input_size=input_size,
                max_steps=max_steps,
                learning_rate=learning_rate,
                hidden_size=hidden_size,
                decoder_output_dim=decoder_output_dim,
                temporal_decoder_dim=temporal_decoder_dim,
                num_encoder_layers=num_encoder_layers,
                num_decoder_layers=num_decoder_layers,
                dropout=dropout,
                batch_size=batch_size,
                hist_exog_list=hist_exog_list,
                val_check_steps=50,
                early_stop_patience_steps=-1
            )
        except Exception as e:
            logger.error(f"  ✗ Model build failed: {e}")
            return 0.0

        # Train
        nf = NeuralForecast(models=[model], freq='D')

        try:
            nf.fit(df=train_inner_df, verbose=False) # verbose False para limpiar salida
        except Exception as e:
            logger.error(f"  ✗ Training failed: {e}")
            return 0.0

        # Rolling predictions con y_prev CORRECTO
        all_predictions = []
        all_true_values = []
        all_prev_values = []

        for i in range(len(valid_df)):
            try:
                if i == 0:
                    hist_df = train_inner_df.copy()
                else:
                    hist_df = pd.concat([train_inner_df, valid_df.iloc[:i]], ignore_index=True)

                forecast = nf.predict(df=hist_df)
                pred = forecast['TiDE'].values[-1]
                true_val = valid_df.iloc[i]['y']

                # ✅ CORRECCIÓN: Obtener y_prev real desde full_df_features
                date = valid_df.iloc[i]['ds']
                try:
                    idx = full_df_features.index.get_loc(pd.Timestamp(date))
                    prev_val = full_df_features['PEN_target'].iloc[idx - 1] if idx > 0 else true_val
                except:
                    prev_val = train_inner_df['y'].iloc[-1] if i == 0 else all_true_values[-1]

                all_predictions.append(pred)
                all_true_values.append(true_val)
                all_prev_values.append(prev_val)

            except Exception as e:
                # Fallback silencioso
                all_predictions.append(np.nan)
                all_true_values.append(valid_df.iloc[i]['y'])
                all_prev_values.append(np.nan)

        # Convertir a arrays y filtrar
        all_predictions = np.array(all_predictions)
        all_true_values = np.array(all_true_values)
        all_prev_values = np.array(all_prev_values)

        valid_mask = ~np.isnan(all_predictions) & ~np.isnan(all_prev_values)
        y_pred_levels = all_predictions[valid_mask]
        y_true_levels = all_true_values[valid_mask]
        y_prev_levels = all_prev_values[valid_mask]

        if len(y_pred_levels) < 10:
            return 0.0

        # ✅ CORRECCIÓN: Usar función corregida para DA
        da = self._calculate_da_corrected(y_true_levels, y_pred_levels, y_prev_levels)

        # Sanity check suave
        if da > 75: da = 0.0 # Filtro anti-overfit extremo

        logger.info(f"  Trial {trial.number}: DA={da:.2f}% (LR={learning_rate:.4f}, Hidden={hidden_size})")
        return da

    def tune(self, train_df_long, full_df_features, hist_exog_list, n_trials, timeout):
        """
        Tuning con base de datos SQLite persistente en Drive.
        """
        logger.info("="*80)
        logger.info(f"HYPERPARAMETER TUNING PERSISTENTE - TiDE h={self.h}")
        logger.info("="*80)

        total_len = len(train_df_long)
        split_idx = int(total_len * 0.8)

        train_inner_df = train_df_long.iloc[:split_idx].copy()
        valid_full_df = train_df_long.iloc[split_idx:].copy()

        n_valid_tuning = min(100, len(valid_full_df))
        valid_df = valid_full_df.iloc[-n_valid_tuning:].copy()

        # --------------------------------------------------------------------
        # 🛡️ CONFIGURACIÓN DE PERSISTENCIA (Storage en Drive)
        # --------------------------------------------------------------------
        # Usamos la carpeta de checkpoints definida globalmente
        db_path = checkpoint.output_dir / "optuna_tuning.db"
        storage_url = f"sqlite:///{db_path}"
        study_name = f"tide_tuning_h{self.h}_corregido"

        logger.info(f"📁 Database: {db_path}")

        # load_if_exists=True permite reanudar el estudio si ya existe en Drive
        study = optuna.create_study(
            study_name=study_name,
            storage=storage_url,
            load_if_exists=True,
            direction='maximize',
            sampler=TPESampler(seed=42) # Usamos seed fija o RANDOM_STATE si está definido
        )

        # Calculamos cuántos trials faltan realmente
        # Contamos trials completados, fallidos o podados
        completed_trials = len([t for t in study.trials if t.state.name in ['COMPLETE', 'FAIL', 'PRUNED']])
        remaining_trials = max(0, n_trials - completed_trials)

        if remaining_trials == 0:
            logger.info("✨ El estudio ya estaba completo. No se ejecutarán más trials.")
        else:
            logger.info(f"↺ Reanudando desde Trial {completed_trials}. Ejecutando {remaining_trials} restantes...")

            try:
                study.optimize(
                    lambda trial: self._objective(trial, train_inner_df, valid_df,
                                                full_df_features, hist_exog_list),
                    n_trials=remaining_trials,
                    timeout=timeout,
                    show_progress_bar=True
                )
            except KeyboardInterrupt:
                logger.warning("\n⚠️ Tuning interrumpido. El progreso se ha guardado en la base de datos.")

        # --- RETORNAR RESULTADOS ---
        valid_trials = [t for t in study.trials if t.value is not None and t.value > 0]

        if len(valid_trials) == 0:
            logger.error("❌ NO VALID TRIALS FOUND IN DB")
            # Devolvemos estructura vacía para no romper el flujo
            return {
                'study': study,
                'best_config': {},
                'best_metrics': {'DA_valid': 0.0},
                'n_valid_trials': 0
            }

        logger.info(f"\nBest DA found: {study.best_value:.2f}%")

        return {
            'study': study,
            'best_config': study.best_params,
            'best_metrics': {'DA_valid': study.best_value},
            'n_valid_trials': len(valid_trials)
        }

print("✓ Celda 6 completada: TiDETuner BLINDADO (SQLite)")

✓ Celda 6 completada: TiDETuner BLINDADO (SQLite)


In [9]:
# ============================================================================
# CELDA 7: ExogSelector (igual estructura, usa DA corregida)
# ============================================================================

class ExogSelector:
    """Selección de exógenas con DA corregida"""

    def __init__(self, h: int = H_FORECAST):
        self.builder = TiDEBuilder()
        self.h = h

    def evaluate_single_exog(self, train_df_long, full_df_features, exog_name, config):
        """Evalúa una exógena con DA corregida"""
        logger.info(f"   Evaluating exog: {exog_name}")

        hist_exog_list = [exog_name] if exog_name != 'NONE' else []

        # Build model
        model = self.builder.build_model(
            h=self.h,
            input_size=config['input_size'],
            max_steps=config['max_steps'],
            learning_rate=config['learning_rate'],
            hidden_size=config['hidden_size'],
            decoder_output_dim=config['decoder_output_dim'],
            temporal_decoder_dim=config['temporal_decoder_dim'],
            num_encoder_layers=config['num_encoder_layers'],
            num_decoder_layers=config['num_decoder_layers'],
            dropout=config['dropout'],
            batch_size=config['batch_size'],
            hist_exog_list=hist_exog_list,
            val_check_steps=50,
            early_stop_patience_steps=-1
        )

        # Split
        total_len = len(train_df_long)
        split_idx = int(total_len * 0.8)
        train_inner = train_df_long.iloc[:split_idx].copy()
        valid = train_df_long.iloc[split_idx:].copy()

        # Train
        nf = NeuralForecast(models=[model], freq='D')

        try:
            nf.fit(df=train_inner)
        except Exception as e:
            logger.error(f"   Training failed: {e}")
            return {'exog': exog_name, 'DA': 0.0, 'MASE': np.inf}

        # Rolling predictions
        all_predictions = []
        all_true_values = []
        all_prev_values = []

        for i in range(len(valid)):
            try:
                if i == 0:
                    hist_df = train_inner.copy()
                else:
                    hist_df = pd.concat([train_inner, valid.iloc[:i]], ignore_index=True)

                forecast = nf.predict(df=hist_df)
                pred = forecast['TiDE'].values[-1]
                true_val = valid.iloc[i]['y']

                # Get y_prev
                date = valid.iloc[i]['ds']
                try:
                    idx = full_df_features.index.get_loc(pd.Timestamp(date))
                    prev_val = full_df_features['PEN_target'].iloc[idx - 1]
                except:
                    prev_val = train_inner['y'].iloc[-1] if i == 0 else all_true_values[-1]

                all_predictions.append(pred)
                all_true_values.append(true_val)
                all_prev_values.append(prev_val)

            except:
                all_predictions.append(np.nan)
                all_true_values.append(valid.iloc[i]['y'])
                all_prev_values.append(np.nan)

        # Calculate metrics with correction
        all_predictions = np.array(all_predictions)
        all_true_values = np.array(all_true_values)
        all_prev_values = np.array(all_prev_values)

        valid_mask = ~np.isnan(all_predictions) & ~np.isnan(all_prev_values)

        if np.sum(valid_mask) < 5:
            return {'exog': exog_name, 'DA': 0.0, 'MASE': np.inf}

        y_pred = all_predictions[valid_mask]
        y_true = all_true_values[valid_mask]
        y_prev = all_prev_values[valid_mask]

        # ✅ CORRECCIÓN: Usar y_prev real
        y_true_ret = np.log(y_true / y_prev)
        y_pred_ret = np.log(y_pred / y_prev)

        dir_true = np.sign(y_true_ret)
        dir_pred = np.sign(y_pred_ret)

        correct = np.sum(dir_true == dir_pred)
        da = 100.0 * correct / len(dir_true)

        mae_ret = np.mean(np.abs(y_true_ret - y_pred_ret))
        # Approximate MASE
        mase = mae_ret / 0.005  # Approximate naive MAE

        logger.info(f"   ✓ {exog_name}: DA={da:.2f}%, MASE={mase:.4f}")

        return {'exog': exog_name, 'DA': da, 'MASE': mase}

    def select_best_exog(self, train_df_long, full_df_features, all_exog_features, config):
        """Selecciona mejor exógena"""
        logger.info("="*80)
        logger.info("EXOGENOUS SELECTION - CORREGIDO")
        logger.info("="*80)

        results = []

        # Evaluar NONE (sin exógenas)
        result_none = self.evaluate_single_exog(train_df_long, full_df_features, 'NONE', config)
        results.append(result_none)

        # Evaluar cada exógena
        for exog in all_exog_features:
            result = self.evaluate_single_exog(train_df_long, full_df_features, exog, config)
            results.append(result)

        # Ordenar por DA (desc), MASE (asc)
        results_sorted = sorted(results, key=lambda x: (-x['DA'], x['MASE']))

        best = results_sorted[0]

        logger.info(f"\n{'='*80}")
        logger.info("SELECTION COMPLETED")
        logger.info(f"{'='*80}")
        logger.info(f"Best exog: {best['exog']} (DA={best['DA']:.2f}%, MASE={best['MASE']:.4f})")

        return {
            'best_exog': best['exog'],
            'best_metrics': {'DA': best['DA'], 'MASE': best['MASE']},
            'all_results': results
        }

logger.info("ExogSelector CORREGIDO loaded")
print("✓ Celda 7 completada: ExogSelector CORREGIDO")


✓ Celda 7 completada: ExogSelector CORREGIDO


In [10]:
# ============================================================================
# CELDA 8: FinalEvaluator (ya estaba correcto)
# ============================================================================

class FinalEvaluator:
    """Evaluación final en holdout (ya usaba y_prev correcto)"""

    def __init__(self, h: int = H_FORECAST):
        self.builder = TiDEBuilder()
        self.h = h

    def train_and_evaluate(self, train_df_long, holdout_df_long, full_df,
                          best_config, best_exog, train_returns):
        """Train en train set y evaluar en holdout"""
        logger.info("="*80)
        logger.info("FINAL EVALUATION - HOLDOUT")
        logger.info("="*80)

        hist_exog_list = [best_exog] if best_exog != 'NONE' else []

        # Build model
        model = self.builder.build_model(
            h=self.h,
            input_size=best_config['input_size'],
            max_steps=best_config['max_steps'],
            learning_rate=best_config['learning_rate'],
            hidden_size=best_config['hidden_size'],
            decoder_output_dim=best_config['decoder_output_dim'],
            temporal_decoder_dim=best_config['temporal_decoder_dim'],
            num_encoder_layers=best_config['num_encoder_layers'],
            num_decoder_layers=best_config['num_decoder_layers'],
            dropout=best_config['dropout'],
            batch_size=best_config['batch_size'],
            hist_exog_list=hist_exog_list,
            val_check_steps=50,
            early_stop_patience_steps=-1
        )

        # Train SOLO en train
        nf = NeuralForecast(models=[model], freq='D')
        nf.fit(df=train_df_long)
        logger.info("✓ TiDE trained (NO LEAKAGE)")

        # Rolling predictions en holdout
        logger.info(f"\n📊 PREDICTING HOLDOUT ({len(holdout_df_long)} obs):")
        all_predictions = []
        all_true_values = []
        all_dates = []

        for i in range(len(holdout_df_long)):
            if i % 10 == 0:
                logger.info(f"   {i+1}/{len(holdout_df_long)}")

            if i == 0:
                hist_df = train_df_long.copy()
            else:
                hist_df = pd.concat([train_df_long, holdout_df_long.iloc[:i]], ignore_index=True)

            try:
                forecast = nf.predict(df=hist_df)
                pred = forecast['TiDE'].values[-1]

                date = holdout_df_long.iloc[i]['ds']
                idx = full_df.index.get_loc(pd.Timestamp(date))
                true_val = full_df['PEN_target'].iloc[idx]

                all_predictions.append(pred)
                all_true_values.append(true_val)
                all_dates.append(date)

            except Exception as e:
                logger.error(f"   Error {i+1}: {e}")
                date = holdout_df_long.iloc[i]['ds']
                idx = full_df.index.get_loc(pd.Timestamp(date))
                true_val = full_df['PEN_target'].iloc[idx]

                pred = all_predictions[-1] if i > 0 else train_df_long['y'].iloc[-1]

                all_predictions.append(pred)
                all_true_values.append(true_val)
                all_dates.append(date)

        y_pred_level = np.array(all_predictions)
        y_true_level = np.array(all_true_values)
        dates = np.array(all_dates)

        logger.info(f"\n✓ {len(y_pred_level)} predictions generated")

        # Calcular y_prev_level
        y_prev_level = []
        for date in dates:
            idx = full_df.index.get_loc(pd.Timestamp(date))
            prev_val = full_df['PEN_target'].iloc[idx - 1] if idx > 0 else full_df['PEN_target'].iloc[idx]
            y_prev_level.append(prev_val)
        y_prev_level = np.array(y_prev_level)

        # Calcular métricas (ya usa y_prev correcto)
        y_true_ret = np.log(y_true_level / y_prev_level)
        y_pred_ret = np.log(y_pred_level / y_prev_level)

        dir_true = np.sign(y_true_ret)
        dir_pred = np.sign(y_pred_ret)

        correct = np.sum(dir_true == dir_pred)
        da = 100.0 * correct / len(dir_true)

        mae_ret = np.mean(np.abs(y_true_ret - y_pred_ret))
        mae_naive = np.mean(np.abs(np.diff(train_returns)))
        mase = mae_ret / mae_naive if mae_naive > 0 else np.inf
        mae_level = np.mean(np.abs(y_true_level - y_pred_level))

        metrics = {
            'DA': da,
            'MASE': mase,
            'MAE_returns': mae_ret,
            'MAE_levels': mae_level
        }

        comparison = FXMetrics.compare_to_baseline(metrics)

        logger.info(f"\n📊 MÉTRICAS HOLDOUT:")
        logger.info(f"  DA: {da:.2f}%")
        logger.info(f"  MASE: {mase:.4f}")
        logger.info(f"\n🎯 VERDICT: {comparison['verdict']}")

        return {
            'best_config': best_config,
            'best_exog': best_exog,
            'holdout_metrics': metrics,
            'comparison': comparison,
            'predictions': {
                'y_true_level': y_true_level,
                'y_pred_level': y_pred_level,
                'y_prev_level': y_prev_level,
                'dates': dates
            },
            'model': nf
        }

logger.info("FinalEvaluator loaded")
print("✓ Celda 8 completada: FinalEvaluator")


✓ Celda 8 completada: FinalEvaluator


In [11]:
# ============================================================================
# CELDA 9: CheckpointManager (VERSIÓN ATÓMICA PRO)
# ============================================================================
import pickle
import tempfile
from pathlib import Path
import logging

logger = logging.getLogger(__name__)

class CheckpointManager:
    """Sistema de checkpointing con Escritura Atómica para Colab"""

    def __init__(self, output_dir: Path = OUTPUT_DIR):
        self.output_dir = Path(output_dir)
        self.checkpoint_dir = self.output_dir / 'checkpoints'
        self.checkpoint_dir.mkdir(parents=True, exist_ok=True)

    def save(self, name: str, obj, force: bool = True):
        """Guarda usando un archivo temporal para evitar corrupción"""
        dest_filepath = self.checkpoint_dir / f"{name}.pkl"

        # Guardamos primero en un temporal .tmp
        temp_filepath = self.checkpoint_dir / f"{name}.tmp"

        try:
            with open(temp_filepath, 'wb') as f:
                pickle.dump(obj, f, protocol=pickle.HIGHEST_PROTOCOL)

            # Reemplazo atómico: Si falla aquí, el archivo original queda intacto
            temp_filepath.replace(dest_filepath)
            # logger.info(f"✓ Checkpoint guardado: {name}") # Comentado para no spamear el log

        except Exception as e:
            logger.error(f"Error guardando checkpoint {name}: {e}")
            if temp_filepath.exists():
                temp_filepath.unlink()
            raise e

        return dest_filepath

    def load(self, name: str):
        filepath = self.checkpoint_dir / f"{name}.pkl"
        if not filepath.exists():
            return None
        try:
            with open(filepath, 'rb') as f:
                logger.info(f"↺ Reanudando desde checkpoint: {name}")
                return pickle.load(f)
        except Exception as e:
            logger.error(f"Archivo corrupto en {name}: {e}")
            return None

    def exists(self, name: str) -> bool:
        return (self.checkpoint_dir / f"{name}.pkl").exists()

    def delete(self, name: str):
        filepath = self.checkpoint_dir / f"{name}.pkl"
        if filepath.exists():
            filepath.unlink()

checkpoint = CheckpointManager()
print("✓ Celda 9 completada: CheckpointManager")

✓ Celda 9 completada: CheckpointManager


In [12]:
# ============================================================================
# CELDA 10: Cargar y Preparar Datos
# ============================================================================

print("="*80)
print("CARGANDO Y PREPARANDO DATOS")
print("="*80)

# Cargar datos
df_raw = pd.read_csv('/content/DATA.csv')
logger.info(f"Data loaded: {df_raw.shape}")

# Feature engineering
engineer = FeatureEngineer()
df_features = engineer.create_features(df_raw)

# Extraer lista de exógenas
all_exog_features = df_features.attrs.get('all_exog_features', [])
print(f"\nTotal exogenous features: {len(all_exog_features)}")

# Clasificar exógenas
exog_to_scale = [f for f in [
    'GARCH_vol', 'MXN_lag_1', 'CLP_lag_1', 'COBRE_lag_1', 'MXPE_lag_1',
    'UST10Y_lag_1', 'DXY_lag_1', 'CPI_lag_1', 'RESERV_lag_1', 'T_TRADE_lag_1', 'Tasa_cds_lag_1'
] if f in all_exog_features]

exog_no_scale = [f for f in [
    'RSI', 'MACD_line', 'month_sin', 'month_cos',
    'day_of_week_sin', 'day_of_week_cos', 'quarter_sin', 'quarter_cos'
] if f in all_exog_features]

print("✓ Celda 10 completada")

CARGANDO Y PREPARANDO DATOS

Total exogenous features: 19
✓ Celda 10 completada


In [13]:
# ============================================================================
# CELDA 11: Split Train/Holdout
# ============================================================================

print("="*80)
print("SPLIT TRAIN/HOLDOUT")
print("="*80)

n_total = len(df_features)
n_train = n_total - N_HOLDOUT
n_holdout = N_HOLDOUT

train_df = df_features.iloc[:n_train].copy()
holdout_df = df_features.iloc[n_train:].copy()

print(f"Total: {n_total} obs")
print(f"Train: {n_train} obs ({n_train/n_total*100:.1f}%)")
print(f"Holdout: {n_holdout} obs ({n_holdout/n_total*100:.1f}%)")
print(f"\nTrain: {train_df.index.min()} to {train_df.index.max()}")
print(f"Holdout: {holdout_df.index.min()} to {holdout_df.index.max()}")

# Train returns para MASE
train_returns = np.log(train_df['PEN_target'] / train_df['PEN_lag_1']).dropna().values

print("✓ Celda 11 completada")

SPLIT TRAIN/HOLDOUT
Total: 7948 obs
Train: 7888 obs (99.2%)
Holdout: 60 obs (0.8%)

Train: 1995-01-19 00:00:00 to 2025-04-14 00:00:00
Holdout: 2025-04-15 00:00:00 to 2025-07-07 00:00:00
✓ Celda 11 completada


In [14]:
# ============================================================================
# CELDA 12: Conversión Long + Scaling
# ============================================================================

print("="*80)
print("CONVERSIÓN LONG + SCALING")
print("="*80)

formatter = NeuralForecastFormatter()

# Convertir a long
train_df_long = formatter.format_to_long(train_df, 'PEN_target', all_exog_features)
holdout_df_long = formatter.format_to_long(holdout_df, 'PEN_target', all_exog_features)

# Fit scaler en train
formatter.fit_scaler(train_df_long, exog_to_scale, exog_no_scale)

# Transform
train_df_long_scaled = formatter.transform_scaler(train_df_long)
holdout_df_long_scaled = formatter.transform_scaler(holdout_df_long)

# Validar
formatter.validate_long_format(train_df_long_scaled)
formatter.validate_long_format(holdout_df_long_scaled)

print(f"Train long shape: {train_df_long_scaled.shape}")
print(f"Holdout long shape: {holdout_df_long_scaled.shape}")

print("✓ Celda 12 completada")


CONVERSIÓN LONG + SCALING
Train long shape: (7888, 22)
Holdout long shape: (60, 22)
✓ Celda 12 completada


In [15]:
# ============================================================================
# CELDA 13: HYPERPARAMETER TUNING - CORREGIDO
# ============================================================================

print("="*80)
print(f"HYPERPARAMETER TUNING TiDE h={H_FORECAST} - CORREGIDO")
print("="*80)

# Verificar checkpoint
if checkpoint.exists('tuning_corregido'):
    print("\n⚡ CHECKPOINT ENCONTRADO - Cargando...")
    tuning_result = checkpoint.load('tuning_corregido')
    print(f"  Best DA: {tuning_result['best_metrics']['DA_valid']:.2f}%")
else:
    print("\n⏱️ Ejecutando tuning corregido (~2 horas)...")
    print(f"Trials: {N_TRIALS}")
    print(f"Timeout: {TIMEOUT}s")

    tuner = TiDETuner(h=H_FORECAST)

    # ✅ CORRECCIÓN: Pasar df_features para acceder a y_prev
    tuning_result = tuner.tune(
        train_df_long=train_df_long_scaled,
        full_df_features=df_features,  # ← NUEVO PARÁMETRO
        hist_exog_list=all_exog_features,
        n_trials=N_TRIALS,
        timeout=TIMEOUT
    )

    # Guardar checkpoint
    checkpoint.save('tuning_corregido', tuning_result, force=True)

print(f"\nBest DA (validation): {tuning_result['best_metrics']['DA_valid']:.2f}%")
print("\nBest Config:")
for key, val in tuning_result['best_config'].items():
    print(f"  {key}: {val}")

print("✓ Celda 13 completada")

HYPERPARAMETER TUNING TiDE h=1 - CORREGIDO

⚡ CHECKPOINT ENCONTRADO - Cargando...
  Best DA: 61.00%

Best DA (validation): 61.00%

Best Config:
  h: 1
  input_size: 168
  max_steps: 300
  learning_rate: 0.0008113524174706105
  hidden_size: 256
  decoder_output_dim: 128
  temporal_decoder_dim: 192
  num_encoder_layers: 4
  num_decoder_layers: 4
  dropout: 0.1
  batch_size: 8
✓ Celda 13 completada


In [16]:
# ============================================================================
# CELDA 14: SELECCIÓN DE EXÓGENAS - CORREGIDA
# ============================================================================

print("="*80)
print("SELECCIÓN DE EXÓGENAS - CORREGIDA")
print("="*80)

if checkpoint.exists('selection_corregido'):
    print("\n⚡ CHECKPOINT ENCONTRADO - Cargando...")
    selection_result = checkpoint.load('selection_corregido')
else:
    print("\n⏱️ Ejecutando selección corregida...")

    selector = ExogSelector(h=H_FORECAST)

    selection_result = selector.select_best_exog(
        train_df_long=train_df_long_scaled,
        full_df_features=df_features,
        all_exog_features=all_exog_features,
        config=tuning_result['best_config']
    )

    checkpoint.save('selection_corregido', selection_result, force=True)

print(f"\nBest Exogenous: {selection_result['best_exog']}")
print(f"DA: {selection_result['best_metrics']['DA']:.2f}%")
print(f"MASE: {selection_result['best_metrics']['MASE']:.4f}")

print("✓ Celda 14 completada")

SELECCIÓN DE EXÓGENAS - CORREGIDA

⚡ CHECKPOINT ENCONTRADO - Cargando...

Best Exogenous: GARCH_vol
DA: 46.01%
MASE: 1.3764
✓ Celda 14 completada


In [17]:
# ============================================================================
# CELDA 15: EVALUACIÓN FINAL EN HOLDOUT
# ============================================================================

print("="*80)
print(f"EVALUACIÓN FINAL EN HOLDOUT TiDE h={H_FORECAST}")
print("="*80)

evaluator = FinalEvaluator(h=H_FORECAST)

final_result = evaluator.train_and_evaluate(
    train_df_long=train_df_long_scaled,
    holdout_df_long=holdout_df_long_scaled,
    full_df=df_features,
    best_config=tuning_result['best_config'],
    best_exog=selection_result['best_exog'],
    train_returns=train_returns
)

print("\n" + "="*80)
print("MÉTRICAS HOLDOUT (TiDE CORREGIDO):")
print(f"  DA: {final_result['holdout_metrics']['DA']:.2f}%")
print(f"  MASE: {final_result['holdout_metrics']['MASE']:.4f}")

print(f"\n📊 COMPARACIÓN vs BASELINES:")
print(f"  vs ARX: ΔDA = {final_result['comparison']['delta_vs_arx']['DA']:+.2f}%")
print(f"  vs NHITS: ΔDA = {final_result['comparison']['delta_vs_nhits']['DA']:+.2f}%")

print(f"\n🎯 VEREDICTO: {final_result['comparison']['verdict']}")

print("✓ Celda 15 completada")


INFO:lightning_fabric.utilities.seed:Seed set to 42
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores


EVALUACIÓN FINAL EN HOLDOUT TiDE h=1


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name                 | Type          | Params | Mode 
---------------------------------------------------------------
0 | loss                 | MAE           | 0      | train
1 | padder_train         | ConstantPad1d | 0      | train
2 | scaler               | TemporalNorm  | 0      | train
3 | hist_exog_projection | MLPResidual   | 1.6 K  | train
4 | dense_encoder        | Sequential    | 1.1 M  | train
5 | dense_decoder        | Sequential    | 725 K  | train
6 | temporal_decoder     | MLPResidual   | 25.1 K | train
7 | global_skip          | Linear        | 169    | train
---------------------------------------------------------------
1.8 M     Trainable params
0         Non-trainable params
1.8 M     Total params
7.372     Total estimated model params size (MB)
66        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]


MÉTRICAS HOLDOUT (TiDE CORREGIDO):
  DA: 45.00%
  MASE: 2.2293

📊 COMPARACIÓN vs BASELINES:
  vs ARX: ΔDA = -6.67%
  vs NHITS: ΔDA = -8.66%

🎯 VEREDICTO: ✗ INFERIOR - Below random walk
✓ Celda 15 completada


In [18]:
# ============================================================================
# CELDA 16: EXPORTAR PREDICCIONES PARA META-LEARNER
# ============================================================================

print("="*80)
print("EXPORTANDO PREDICCIONES PARA META-LEARNER")
print("="*80)

# Crear directorio de predicciones
predictions_dir = Path('/content/drive/MyDrive/Colab_Outputs/predictions_dump')
predictions_dir.mkdir(parents=True, exist_ok=True)

# Crear DataFrame de exportación
preds = final_result['predictions']

df_export = pd.DataFrame({
    'ds': preds['dates'],
    'y_pred': preds['y_pred_level'],
    'model': 'TiDE',
    'type': 'levels'
})

# Guardar
export_path = predictions_dir / 'pred_TiDE.csv'
df_export.to_csv(export_path, index=False)

print(f"\n✓ Predicciones exportadas: {export_path}")
print(f"  Filas: {len(df_export)}")
print(f"  Columnas: {df_export.columns.tolist()}")
print(f"\nPrimeras 5 filas:")
print(df_export.head())

# Verificación
print(f"\n📊 VERIFICACIÓN:")
print(f"  N_HOLDOUT: {len(df_export)} (esperado: 60)")
print(f"  NaNs: {df_export['y_pred'].isna().sum()}")
print(f"  Type: {df_export['type'].unique()[0]}")

print("✓ Celda 16 completada")


EXPORTANDO PREDICCIONES PARA META-LEARNER

✓ Predicciones exportadas: /content/drive/MyDrive/Colab_Outputs/predictions_dump/pred_TiDE.csv
  Filas: 60
  Columnas: ['ds', 'y_pred', 'model', 'type']

Primeras 5 filas:
          ds    y_pred model    type
0 2025-04-15  3.709218  TiDE  levels
1 2025-04-16  3.704835  TiDE  levels
2 2025-04-17  3.716212  TiDE  levels
3 2025-04-18  3.727357  TiDE  levels
4 2025-04-21  3.737954  TiDE  levels

📊 VERIFICACIÓN:
  N_HOLDOUT: 60 (esperado: 60)
  NaNs: 0
  Type: levels
✓ Celda 16 completada


In [19]:
# ============================================================================
# CELDA 17: GUARDAR MÉTRICAS Y CONFIG
# ============================================================================

print("="*80)
print("GUARDANDO MÉTRICAS Y CONFIGURACIÓN")
print("="*80)

# Guardar métricas
metrics_to_save = {
    'model': 'TiDE_CORREGIDO',
    'horizon': 'h=1',
    'holdout_metrics': final_result['holdout_metrics'],
    'best_config': final_result['best_config'],
    'best_exog': final_result['best_exog'],
    'comparison': {
        'vs_ARX': final_result['comparison']['delta_vs_arx'],
        'vs_NHITS': final_result['comparison']['delta_vs_nhits']
    },
    'verdict': final_result['comparison']['verdict'],
    'bug_fix': 'DA calculation corrected to use y_prev real instead of np.diff()',
    'timestamp': datetime.now().isoformat()
}

metrics_path = OUTPUT_DIR / 'metrics.json'
with open(metrics_path, 'w') as f:
    json.dump(metrics_to_save, f, indent=2, default=str)

print(f"✓ Métricas guardadas: {metrics_path}")

# Guardar modelo
model_path = OUTPUT_DIR / 'tide_h1_corregido.pkl'
with open(model_path, 'wb') as f:
    pickle.dump(final_result['model'], f)

print(f"✓ Modelo guardado: {model_path}")

print("✓ Celda 17 completada")

GUARDANDO MÉTRICAS Y CONFIGURACIÓN
✓ Métricas guardadas: /content/drive/MyDrive/Colab_Outputs/TiDE_h1_USD_PEN_CORREGIDO/metrics.json
✓ Modelo guardado: /content/drive/MyDrive/Colab_Outputs/TiDE_h1_USD_PEN_CORREGIDO/tide_h1_corregido.pkl
✓ Celda 17 completada


In [20]:
# ============================================================================
# RESUMEN FINAL
# ============================================================================

print("\n" + "="*80)
print("🎉 ¡EXPERIMENTO TiDE h=1 CORREGIDO COMPLETADO!")
print("="*80)

print(f"\n📊 RESULTADOS FINALES:")
print(f"  DA: {final_result['holdout_metrics']['DA']:.2f}%")
print(f"  MASE: {final_result['holdout_metrics']['MASE']:.4f}")

print(f"\n⚠️ CORRECCIÓN APLICADA:")
print(f"  Bug: np.diff() calculaba cambios internos de cada serie")
print(f"  Fix: Usar y_prev real para calcular retornos")
print(f"  Resultado: DA ahora refleja precisión direccional REAL")

print(f"\n📁 ARCHIVOS GENERADOS:")
print(f"  • {OUTPUT_DIR / 'metrics.json'}")
print(f"  • {OUTPUT_DIR / 'tide_h1_corregido.pkl'}")
print(f"  • {predictions_dir / 'pred_TiDE.csv'}")

print(f"\n🔧 PRÓXIMOS PASOS:")
print(f"  1. Comparar con versión anterior (DA=41.67%)")
print(f"  2. Si DA < 50%, considerar que TiDE no es apto para USD/PEN h=1")
print(f"  3. Continuar con meta-learner usando predicciones exportadas")

print("="*80)


🎉 ¡EXPERIMENTO TiDE h=1 CORREGIDO COMPLETADO!

📊 RESULTADOS FINALES:
  DA: 45.00%
  MASE: 2.2293

⚠️ CORRECCIÓN APLICADA:
  Bug: np.diff() calculaba cambios internos de cada serie
  Fix: Usar y_prev real para calcular retornos
  Resultado: DA ahora refleja precisión direccional REAL

📁 ARCHIVOS GENERADOS:
  • /content/drive/MyDrive/Colab_Outputs/TiDE_h1_USD_PEN_CORREGIDO/metrics.json
  • /content/drive/MyDrive/Colab_Outputs/TiDE_h1_USD_PEN_CORREGIDO/tide_h1_corregido.pkl
  • /content/drive/MyDrive/Colab_Outputs/predictions_dump/pred_TiDE.csv

🔧 PRÓXIMOS PASOS:
  1. Comparar con versión anterior (DA=41.67%)
  2. Si DA < 50%, considerar que TiDE no es apto para USD/PEN h=1
  3. Continuar con meta-learner usando predicciones exportadas
